# Preliminary of hack-phenomx env and inital development tasks
* Compare PhenomXHM modes to NR data
  * Load NR data
  * Generate model waveform 
  * Compare by plotting FD waveforms (different modes)
* Apply test modifation to PhenomXHM ringdown frequency and compare to NR and default PhenomXHM

In [1]:
# Setup ipython environment
%load_ext autoreload
%autoreload 2
%matplotlib inline
# Setup the notebook's environment
import lalsimulation as lalsim
import lal
import numpy as np
from numpy import *
from matplotlib.pyplot import *
from positive import *
from nrutils import lalphenom
from positive.units import *
from nrutils import scsearch,gwylm
from xcp import *
import warnings

(positive)>> Applying custom matplotlib settings.
(init)>> Warm greetings from XCP.
(xcp.core)>> Catalog of calibration runs stored to "xcp.calibration_catalog"
(xcp.core)>> Metadata dictionary for Ed's catalog paper stored to "xcp.catalog_paper_metadata"


In [2]:
# Let the user know where lalsimulation lives
alert('We are getting our LALSimulation from:\n%s'%green(lalsim.__path__[0]))

(notebook)>> We are getting our LALSimulation from:
/Users/book/opt/anaconda2/envs/hack-phenomx3/lib/python3.6/site-packages/lalsimulation


In [3]:
# error('This notebook is made for the o773b-ll branch of Jonathan Thompsons lalsuite fork. You must have checkout and installed lalsuite from this branch before running this notebook. ')

AssertionError: [1m([31merror!![0m)[0m>> This notebook is made for the o773b-ll branch of Jonathan Thompsons lalsuite fork. You must have checkout and installed lalsuite from this branch before running this notebook. 

## Load reference NR data

In [ ]:
# Find a catalog entry of interest using a keyword search
# a = scsearch(keyword='q1a02t120',verbose=True)[0]
a = scsearch(keyword='q1a02t60_T_96_384',verbose=True)[0]
# a = scsearch(keyword='q8a08t150dP2_T_120_480',verbose=True)[0]

In [ ]:
# Load the realted NR waveform
y = {}
lpm_space = [ (2,2),(2,1),(2,0)] # ,(3,3),(3,2),(4,4)
lm_space = list(set(lpm_space + [ (l,-m) for l,m in lpm_space  ]))
y['raw'] = gwylm(a,lm=lm_space,dt=0.5,clean=True,verbose=False,pad=2000, enforce_m_relative_sign_convention=not False)

In [ ]:
# Generate waveform in L-frame (this rotates all multipoles and vector metadata)
# y['j'] = y['raw'].__calc_j_of_t_frame__()
y['j'] = y['raw'].__calc_initial_j_frame__()

In [ ]:
# Generate waveform in L-frame (this rotates all multipoles and vector metadata)
y['lal'] = y['j'].__calc_initial_l_frame__()

In [ ]:
y['cp'] = y['j'].__calc_coprecessing_frame__(transform_domain='td',kind='psi4')

In [ ]:
y['cp-sym'] = y['cp'].__symmetrize__(zparity=True)

In [ ]:
y['cp'][2,2]['psi4'].plot()

## Compute model waveform for comparison

In [ ]:

def PolarizationsFD(approximant, modeList, m1, m2, s1, s2, delta_f, phiRef,nu0 = 0,pflag=501, ReturnCoPrec=1):
      
    lalparams = lal.CreateDict()
    
    #
    output_modes = {}
    
    #
    ModeArray = lalsim.SimInspiralCreateModeArray()
    for mode in modeList:
        
        #
        l,m = mode
        
        #
        lalsim.SimInspiralModeArrayActivateMode(ModeArray, l,m)
        lalsim.SimInspiralWaveformParamsInsertModeArray(lalparams, ModeArray)

        #
        lalsim.SimInspiralWaveformParamsInsertPhenomXHMThresholdMband(lalparams, 0)

        #
        lalsim.SimInspiralWaveformParamsInsertPhenomXReturnCoPrec(lalparams, ReturnCoPrec)
        
        #
        if pflag:
            lalsim.SimInspiralWaveformParamsInsertPhenomXPrecVersion( lalparams, pflag )

        #
        f_min       = 10.0
        f_max       = 2048.0
        Omega       = 0.
        inclination = 0 # Chosen so that it doesn't correxpond to a spherical hamonic root
        distance_Mpc= 100.0
        distance    = distance_Mpc*1.0e6*lal.PC_SI

        Hp, Hc = lalsim.SimInspiralChooseFDWaveform(m1=lal.MSUN_SI*m1,
                                                m2=lal.MSUN_SI*m2, 
                                                S1x=s1[0], S1y=s1[1], S1z=s1[2],
                                                S2x=s2[0], S2y=s2[1], S2z=s2[2],
                                                distance=distance, 
                                                inclination=inclination, 
                                                LALpars=lalparams,
                                                phiRef=phiRef, 
                                                f_ref=f_min,
                                                deltaF=delta_f,
                                                f_min=f_min,
                                                f_max=f_max,
                                                longAscNodes=Omega,
                                                eccentricity=0.0,
                                                meanPerAno=0.0,
                                                approximant=approximant) 

        #
        freqs = np.arange(len(Hp.data.data)) * delta_f
        hp = Hp.data.data
        hc = Hc.data.data
        
        if not ( approximant in (lalsim.IMRPhenomXP,lalsim.IMRPhenomXPHM) ):

            #
            s = -2
            spherical_harmonic = sYlm(s,l,m,inclination,phiRef)
            hp /= spherical_harmonic
            hc /= spherical_harmonic
        
        #
        Mtot = m1+m2
        hp = codehf(hp,Mtot,distance_Mpc)
        hc = codehf(hc,Mtot,distance_Mpc)
        f  = codef(freqs,Mtot) 
        
        #
        output_modes[l,m] = (hp,hc,f)
    
    #
    return output_modes

In [ ]:

l,m = 2,2
modeList = [ [l,m] ]
pflag = 500
M = 100

# m1 = y['lal'].m1 * M / ( y['lal'].m1 + y['lal'].m2 )
# m2 = y['lal'].m2 * M / ( y['lal'].m1 + y['lal'].m2 )
# s1 = [0,0,y['lal'].X1[-1]] # Only use z-component becuase PhenomXHM (no precession) is the approx
# s2 = [0,0,y['lal'].X2[-1]] # Only use z-component becuase PhenomXHM (no precession) is the approx

m1 = a.m1 * M / ( a.m1 + a.m2 )
m2 = a.m2 * M / ( a.m1 + a.m2 )
nonprec_s1 = [0,0,y['lal'].X1[-1]] # Only use z-component becuase PhenomXHM (no precession) is the approx
nonprec_s2 = [0,0,y['lal'].X2[-1]] # Only use z-component becuase PhenomXHM (no precession) is the approx
s1 = y['lal'].X1 #
s2 = y['lal'].X2 # 

delta_f = 0.125
phiRef = 0

#
h     = {}
ls    = {}
lw    = {}
alpha = {}


#
key = 'XAS'
moments_dict_a = PolarizationsFD(lalsim.IMRPhenomXAS, modeList, m1, m2, nonprec_s1, nonprec_s2, delta_f, phiRef, pflag=0, ReturnCoPrec=1)
hp,hc,f    = moments_dict_a[l,m]
h[key]     = hp + 1j*hc
ls[key]    = '-'
lw[key]    = 1
alpha[key] = 0.3

#
key = 'XP-CP-Sz-Only'
moments_dict_b = PolarizationsFD(lalsim.IMRPhenomXP, modeList, m1, m2, nonprec_s1, nonprec_s2, delta_f, phiRef, pflag=0, ReturnCoPrec=1)
hp,hc,f    = moments_dict_b[l,m]
h[key]     = hp + 1j*hc
ls[key]    = ':'
lw[key]    = 4
alpha[key] = 1

#
key = 'XP-CP'
moments_dict_1 = PolarizationsFD(lalsim.IMRPhenomXP, modeList, m1, m2, s1, s2, delta_f, phiRef, pflag=0, ReturnCoPrec=1)
hp,hc,f    = moments_dict_1[l,m]
h[key]     = hp + 1j*hc
ls[key]    = '-'
lw[key]    = 1
alpha[key] = 1

#
key = 'XP-CP-501'
moments_dict_1 = PolarizationsFD(lalsim.IMRPhenomXP, modeList, m1, m2, s1, s2, delta_f, phiRef, pflag=501, ReturnCoPrec=1)
hp,hc,f    = moments_dict_1[l,m]
h[key]     = hp + 1j*hc
ls[key]    = '-'
lw[key]    = 1
alpha[key] = 1

# #
# key = 'XP-CP-502'
# moments_dict_2 = PolarizationsFD(lalsim.IMRPhenomXP, modeList, m1, m2, s1, s2, delta_f, phiRef, pflag=502, ReturnCoPrec=1)
# hp,hc,f    = moments_dict_2[l,m]
# h[key]     = hp + 1j*hc
# ls[key]    = '-'
# lw[key]    = 1
# alpha[key] = 1

In [ ]:
# Compare single mode generated above to NR

#
figure( figsize=6*figaspect(0.618*2) )


# --- #
a1=subplot(2,1,1)
# --- #


# plot( y['lal'].f,y['cp'][l,m]['strain'].fd_amp,label='nr-cp-frame',lw=lw,ls='--' )
plot( y['lal'].f,y['cp-sym'][l,m]['strain'].fd_amp,label='nr-cp-sym-frame',lw=6,ls='-',color='k',alpha=0.2 )
# plot( y['lal'].f,y['lal'][l,m]['strain'].fd_amp,label='nr-lal-frame',lw=lw )

#
for key in h:
    plot( f, abs(h[key]),label=key,lw=lw[key],ls=ls[key],alpha=alpha[key] )
    
# plot( f, abs(h1),label='PhenomXP-with-wRDEff',lw=lw, color='r' )
# plot( f, abs(h2),label='PhenomXP-With-SxSySz',lw=lw )
# plot( f, abs(ha),label='PhenomXAS',lw=1, color='k', alpha=0.16, ls='-' )
# plot( f, abs(hb),label='PhenomXP-With-Sz',lw=4, color='r', alpha=0.36, ls=':' )

xscale('log')
yscale('log')
xlim(1.5*y['cp'].wstart/(2*3.141592659),0.12)

mask = (y['lal'].f>min(xlim())) & (y['lal'].f<max(xlim()))
ylim( lim(y['cp-sym'][l,m]['strain'].fd_amp[mask], dilate=1.1, dilate_with_multiply=True) )

legend()
xlabel('$f M$')
ylabel(r'$|h_{%i%i}|$'%(l,m))
title(y['lal'].simname)


# --- #
subplot(2,1,2,sharex=a1)
# --- #

#
f0,f1 = 0.02,0.1
def calc_dphi(f,h,use_smooth=False):
    phi = np.unwrap(np.angle(h))
    dphi = spline_diff(f,phi)
    dphi -= np.mean(dphi[ (f>f0) & (f<f1) ])
    return smooth(dphi,width=20).answer if use_smooth else dphi


# plot( y['lal'].f,calc_dphi(y['lal'].f,y['cp'][l,m]['strain'].fd_y,use_smooth=True),label='nr-cp-frame',lw=lw,ls='--' )
plot( y['lal'].f,calc_dphi(y['lal'].f,y['cp-sym'][l,m]['strain'].fd_y,use_smooth=True),label='nr-cp-sym-frame',lw=6,ls='-',color='k',alpha=0.2 )
# plot( y['lal'].f,calc_dphi(y['lal'].f,y['lal'][l,m]['strain'].fd_y),label='nr-lal-frame',lw=lw )

#
for key in h:
    plot( f, calc_dphi(f,h[key]),label=key,lw=lw[key],ls=ls[key],alpha=alpha[key] )
    
# pi = 3.14159265359
# plot( f, calc_dphi(f,h1),label='PhenomXP-with-wRDEff',lw=lw, color='r' )
# plot( f, calc_dphi(f,h2),label='PhenomXP-With-SxSySz',lw=lw )
# plot( f, calc_dphi(f,ha),label='PhenomXAS',lw=1, color='k', alpha=0.16, ls='-' )
# plot( f, calc_dphi(f,hb),label='PhenomXP-With-Sz',lw=4, color='r', alpha=0.36, ls=':' )

xscale('log')
mask = (f>f0) & (f<f1)
ylim( lim(calc_dphi(f,h[key])[mask],dilate=.1) )
legend()
xlabel('$f M$')
ylabel(r'$\partial_{f}\;{\phi_{%i%i}}$'%(l,m))
title(y['lal'].simname)


# --- #
savefig('/Users/book/Desktop/0_preliminaries_check_%s.png'%y['lal'].simname)
# --- #
